In [1]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator

In [2]:
patches = {'pz_only' : '/home/rt2122/Data/Planck/coords/pz_only_patches.csv', 
          'act_erosita' : '/home/rt2122/Data/Planck/coords/act_erosita_patches.csv',
          'pz_act_jan' : '/home/rt2122/Data/Planck/coords/pz_act_jan_patches.csv',
          'pz_act_found' : '/home/rt2122/Data/Planck/coords/pz_act_found_patches.csv',
          'pz_all_found' : '/home/rt2122/Data/Planck/coords/pz_all_found_patches.csv'}

In [3]:
pd.read_csv(patches['pz_only'])

,x,y,pix
0,720,438,0
1,780,489,0
2,491,386,0
3,59,729,0
4,647,899,0
...,...,...,...
3665328,570,494,47
3665329,262,149,47
3665330,368,545,47
3665331,74,98,47


In [4]:
def patches_df_get_ra_dec(df):
    from DS_healpix_fragmentation import one_pixel_fragmentation, pix2radec
    df['pix11'] = 0
    for i in tqdm(range(48)):
        matr = one_pixel_fragmentation(2, i, 10)
        x = df[df['pix'] == i]['x']
        y = df[df['pix'] == i]['y']
        df['pix11'].loc[df['pix'] == i] = matr[x, y]
    df['RA'], df['DEC'] = pix2radec(df['pix11'], 2**11)
    return df

In [5]:
def stat_patches_file(file_name, true_cats_dir='/home/rt2122/Data/original_catalogs/csv/'):
    import os
    import numpy as np
    import pandas as pd
    from DS_healpix_fragmentation import pix2radec
    from astropy.coordinates import SkyCoord
    from astropy import units as u
    
    df = pd.read_csv(file_name)
    df['x'] += 32
    df['y'] += 32
    df = patches_df_get_ra_dec(df)
    line = {}
    line['number of patches'] = len(df)
    
    df_sc = SkyCoord(ra=np.array(df['RA'])*u.degree, dec=np.array(df['DEC'])*u.degree, frame='icrs')
    
    true_cats_files = next(os.walk(true_cats_dir))[-1]
    true_cats_files = [os.path.join(true_cats_dir, file) for file in true_cats_files]
    
    true_cats = {os.path.splitext(os.path.basename(file))[0] : pd.read_csv(file) for file in true_cats_files}
    
    for tr_name in true_cats:
        tr = true_cats[tr_name]
        tr_sc = SkyCoord(ra=np.array(tr['RA'])*u.degree, dec=np.array(tr['DEC'])*u.degree, frame='icrs')
        
        _, d2d, _ = df_sc.match_to_catalog_sky(tr_sc)
        line[tr_name + ' count'] = np.count_nonzero(d2d.degree < 1.83)
    return line

In [6]:
a = np.array([[1, 2, 3], [4, 5, 6], [1, 5, 6]])
x = [1, 2, 0]
y = [0, 0, 2]
a, a[x, y]

(array([[1, 2, 3],
        [4, 5, 6],
        [1, 5, 6]]),
 array([4, 1, 3]))

In [8]:
df = pd.read_csv(patches['pz_only'])

In [16]:
patches_df_get_ra_dec(df)

,x,y,pix,pix11
0,720,438,0,648980
1,780,489,0,742625
2,491,386,0,256142
3,59,729,0,285643
4,647,899,0,901167
...,...,...,...,...
3665328,570,494,47,0
3665329,262,149,47,0
3665330,368,545,47,0
3665331,74,98,47,0


In [6]:
df = []
for name in patches:
    df.append(pd.DataFrame(stat_patches_file(patches[name]), index=[name]))
df = pd.concat(df)
df

,number of patches,PSZ2 count,MCXC count,RM count,ACT count,Abell count
pz_only,3665333,3665290,2463887,1581862,1625157,2608799
act_erosita,6948027,2726346,2832677,3613607,6948014,5118436
pz_act_jan,2440053,1385149,1187728,1176940,1929939,1751454
pz_act_found,1323071,1030801,764449,603087,813073,952408
pz_all_found,1865878,1440335,1023350,738879,962807,1239383


In [7]:
df.to_csv('/home/rt2122/Data/patches_stat.csv')

In [8]:
for name in ['PSZ2', 'MCXC', 'RM', 'ACT', 'Abell']:
    df[name + ' coef'] = df[name + ' count'] / df['number of patches']

In [9]:
df_coef = df[['number of patches', 'PSZ2 coef',
 'MCXC coef',
 'RM coef',
 'ACT coef',
 'Abell coef']]

In [10]:
cm = sns.light_palette("green", as_cmap=True)

s = df_coef.style.background_gradient(cmap=cm)
s

,number of patches,PSZ2 coef,MCXC coef,RM coef,ACT coef,Abell coef
pz_only,3665333,0.999988,0.672214,0.431574,0.443386,0.711750
act_erosita,6948027,0.392391,0.407695,0.520091,0.999998,0.736675
pz_act_jan,2440053,0.567672,0.486763,0.482342,0.790941,0.717793
pz_act_found,1323071,0.779097,0.577784,0.455824,0.614535,0.719846
pz_all_found,1865878,0.771934,0.548455,0.395995,0.516007,0.664236
